# Lock exchange flow example
This is a test case with particles released in a simple looping flow field 
The background flow velocity is given by a lock exchange flow.

In [1]:
# Load required libraries
using Pkg
Pkg.instantiate()
Pkg.activate("..")
using Particles
using Plots
gr()
using Random

randpool = MersenneTwister(0) #Generate same random numbers every time

nothing

  Activating project at `e:\Master_Thesis\particles.jl`


In [34]:
dflow_map = load_nc_info(@__DIR__, r"locxx_map.nc")
dflow_map.variables

┌ Info: e:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc
└ @ Particles e:\Master_Thesis\particles.jl\src\dflow.jl:27


ErrorException: type Array has no field variables

In [17]:
# load useful default settings 
d=default_userdata()
n=10 #number of particles

# general setting
d["coordinates"] = "projected"
d["time_direction"] = :forwards # :forwards or :backwards
d["variables"] = ["x","y", "z", "age"]

# load data from nc file 
dflow_map = load_nc_info(@__DIR__, r"locxx_map.nc")
# construct a grid for interpolation
# 2nd argument is number of grid points per dimension
# 3rd argument to supress spherical coordinates
interp = load_dflow_grid(dflow_map, 50, false); 

# read the reftime from the file
d["reftime"] = get_reftime(dflow_map)
t_ref = get_reftime(dflow_map)
rel_times = get_times(dflow_map, t_ref)
d["dt"] = rel_times[2] - rel_times[1]

# plot maps
d["plot_maps_times"] = rel_times                        # Time at which plot should be made
d["plot_maps"] = true

# particle settings
d["nparticles"]= 10 #number of particles
p0 = zeros(length(d["variables"]), length(d["nparticles"]))
p0[1, :] = 


compute index:
- e:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc


┌ Info: e:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc
└ @ Particles e:\Master_Thesis\particles.jl\src\dflow.jl:27


true

In [1]:
display(dflow_map)
display(d["variables"])

UndefVarError: UndefVarError: dflow_map not defined

In [19]:
# # check default reftime
# println(d["reftime"])
# # read reftime and times using lib functions
# t_ref = get_reftime(dflow_map)
# println(t_ref)
# println(d["reftime"] == t_ref)
# times = get_times(dflow_map, t_ref)
# # read times, equivalent to using lib functions
# time_relative = dflow_map[1].vars["time"]
# units = time_relative.atts["units"]
# println(time_relative == times)
# println(time_relative == dflow_map[1]["time"][:])

In [20]:
# prepare the interpolator for velocity field
# no need for the 3rd dimension 
u = initialize_interpolation(dflow_map, interp, "mesh2d_ucx", d["reftime"], 0.0, d["time_direction"]);
v = initialize_interpolation(dflow_map, interp, "mesh2d_ucy", d["reftime"], 0.0, d["time_direction"]);
display(typeof(u))


Particles.var"#f#20"{Interpolator, Float64, Symbol, Particles.var"#weights#19", Particles.var"#update_cache_backwards#17"{Vector{Any}}, Particles.var"#update_cache_forwards#14"{Vector{Int64}, Vector{Any}, Bool, Int64}, Vector{Any}}

initialize caching for e:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucx...
initialize caching for e:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucy...


In [21]:
# explicitly prepare the grid for ploting
scale = 10
x_resolution=31 #[0,300]
y_resolution=31   #[0,1]
x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))

xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)
Plots.default(:size, d["plot_maps_size"])
u_plot = zeros(length(x)*length(y))
v_plot = zeros(length(x)*length(y))
for i = 1:length(x)
    for j = 1:length(y)
        u_plot[(i-1)*length(y)+j] = u(x[i], y[j], 0.0, 30.0)
        v_plot[(i-1)*length(y)+j] = v(x[i], y[j], 0.0, 30.0)
    end
end
quiver(xs,ys,quiver=(scale*u_plot,scale*v_plot))
savefig("bkimg.png")


"e:\\Master_Thesis\\particles.jl\\case_lock_exchange\\bkimg.png"

In [22]:
###### Velocity function for the particles ######
function f1!(ds, s, t, i, d)
   x, y, z, age = s
   z = 0.0
   up = 0
   vp = 0
   dt = d["dt"]
   uw = u(x, y, z, t)
   vw = v(x, y, z, t)

   # The model with :
   # 1: Only flow velocities
   up = uw
   vp = vw
   # original computation for computing the particle velocity
   #(K, Kdx, Kdy) = estimate_viscosity_smag(interp, x, y, t, u, v)
   #if !(uw == vw == ua == va == 0.0)
   #   # https://doi.org/10.1016/j.ocemod.2017.11.008 eq. 27
   #   up += Kdy + randn() * sqrt(2 * K * dt) / dt
   #   vp += Kdx + randn() * sqrt(2 * K * dt) / dt
   #end
   
   epsx = 0#1
   epsz = 0#1e-5
   up += randn()*sqrt(2*epsx*dt)/dt
   vp += randn()*sqrt(2*epsz*dt)/dt

   ds.x = up
   ds.y = vp
   ds.z = 0.0
   ds.t = 1.0


   if d["time_direction"] == :backwards
      up *= -1
      vp *= -1
   end
end
d["f"]=f1!

f1! (generic function with 1 method)

In [23]:
# use streamfunction as background for plotting
function plot_background(d)
    scale = 10
    x_resolution=91 #[0,300]
    y_resolution=31   #[0,1]
    x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
    y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))
    xs=kron(x,ones(length(y)))
    ys=kron(ones(length(x)),y)
    Plots.default(:size, d["plot_maps_size"])
    u_plot = u.(xs, ys, 0, 0)
    v_plot = v.(xs, ys, 0, 0)
    f = quiver(xs,ys,quiver=(scale*u_plot,scale*v_plot))
    return(f)
 end
 d["plot_maps_background"] = plot_background
 

plot_background (generic function with 1 method)

In [24]:
#run the actual particle model
@time run_simulation(d)

plotting output at t=Any[0.0, 30.00000000000005, 59.99999999999979, 89.9999999999995, 119.99999999999922, 149.99999999999997, 180.0000000000011, 210.00000000000225, 240.00000000000338, 270.0000000000045  …  330.00000000000676, 360.0000000000079, 390.00000000000904, 420.0000000000102, 450.0000000000113, 480.00000000001245, 510.0000000000136, 540.0000000000094, 570.0000000000049, 600.0]


MethodError: MethodError: no method matching getindex(::Int64, ::Int64, ::Colon)
Closest candidates are:
  getindex(::Number, ::Integer) at number.jl:96
  getindex(::Number, !Matched::Integer...) at number.jl:101
  getindex(::Number) at number.jl:95
  ...

In [ ]:
function plot_maps_xy(fig, d, p)
    if "x" in d["variables"]
        x_index = index("x", d["variables"])
        y_index = index("y", d["variables"])
    elseif "lon" in d["variables"]
        x_index = index("lon", d["variables"])
        y_index = index("lat", d["variables"])
    else
        error("plot_maps_xy: no spatial variables x,y or lat,lon found")
    end
    scatter!(fig, p[x_index, :], p[y_index, :], markercolor = :black, legend = false)
end

In [25]:
# # screenshot of particles
# t_plot=d["keep_particle_times"]
# p=d["all_particles"]
# for i in 1:length(t_all)
#     fig = d["plot_maps_background"](d)
#     fname = "vortex\\screenshot_time"*"$(t_plot[i])"*".png"
#     d["plot_maps_func"](fig, d, p[i])
#     savefig(fig, fname)
# end


In [26]:
# obtain the location data
display(d["variables"])
display(length(t_plot))
display("Particle number")
display(n)

4-element Vector{String}:
 "x"
 "y"
 "z"
 "age"

UndefVarError: UndefVarError: t_plot not defined

In [27]:
# p_x = zeros(length(t_plot), n)
# p_y = zeros(length(t_plot), n)
# for i in 1:length(t_plot)
#     p_x[i, :] = p[i][1,:]
#     p_y[i, :] = p[i][2,:]
# end

# # prepare the data for plotting
# # note that the data points at the first and the end are used once
# indices = reduce(vcat, [[j for i in 1:2] for j in 1:length(t_plot)])
# indices = indices[begin+1:end-1]
# fig = d["plot_maps_background"](d)
# dname = "loccx.png"

# for i in 1:n
#     plot!(fig, p_x[indices, i], p_y[indices, i], legend=egend =:outerright, size = (1400, 800))
# end

# savefig(fig, fname)
